<a href="https://colab.research.google.com/github/Nataliya2401/DS_11/blob/main/hw_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, accuracy_score, classification_report
from sklearn import preprocessing
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
path = '/content/drive/MyDrive/Colab Notebooks/hw_5/data'

In [4]:
folders = listdir(path)
folders

['running', 'stairs', 'idle', 'walking']

In [5]:
for f in folders:
  frame = join(path, f)
  frames = listdir(frame)
  print(f, '::', len(frames))

running :: 3408
stairs :: 165
idle :: 1039
walking :: 1850


In [6]:
ff = listdir(join(path,'stairs' ))
ff[5]
df_t = pd.read_csv(join(join(path,'stairs'), ff[5]))
df_t.sample(3)

,accelerometer_X,accelerometer_Y,accelerometer_Z
7,-0.330400,-6.105214,-3.284845
13,-3.102885,-4.644751,-2.456451
27,-2.312799,-15.825673,9.706094


### Create function for calculating statistic features

In [7]:
def create_prepared_df (df, type_activity):
  df_prepared = pd.DataFrame({'min_X': df['accelerometer_X'].min(),
                              'mean_X': df['accelerometer_X'].mean(),
                              'median_X': df['accelerometer_X'].median(),
                              'max_X': df['accelerometer_X'].max(),
                              'std_X': df['accelerometer_X'].std(),
                              'variance_X': df['accelerometer_X'].var(),
                              'skew_X': df['accelerometer_X'].skew(),

                              'min_Y': df['accelerometer_Y'].min(),
                              'mean_Y': df['accelerometer_Y'].mean(),
                              'median_Y': df['accelerometer_Y'].median(),
                              'max_Y': df['accelerometer_Y'].max(),
                              'std_Y': df['accelerometer_Y'].std(),
                              'variance_Y': df['accelerometer_Y'].var(),
                              'skew_Y': df['accelerometer_Y'].skew(),

                              'min_Z': df['accelerometer_Z'].min(),
                              'mean_Z': df['accelerometer_Z'].mean(),
                              'median_Z': df['accelerometer_Z'].median(),
                              'max_Z': df['accelerometer_Z'].max(),
                              'std_Z': df['accelerometer_Z'].std(),
                              'variance_Z': df['accelerometer_Z'].var(),
                              'skew_Z': df['accelerometer_Z'].skew(),
                              'activity': type_activity
                            }, index=[0])
  return df_prepared
# pd.DataFrame({'a': 1, 'b': 2}, index=[0])

In [8]:
def read_data_for_prepared_df(folder):

  df = pd.DataFrame()
  for item in folders:
    folder_path = join(path, item)

    for filename in listdir(folder_path):
      if filename.endswith('.csv'):
        file_path = join(folder_path, filename)
        df_temp = pd.read_csv(file_path)
        df_temp = create_prepared_df(df_temp, item)
        df = pd.concat([df, df_temp], ignore_index=True)

  return df


In [9]:
df_new = read_data_for_prepared_df(folders)

In [10]:
df_new.sample(5)

,min_X,mean_X,median_X,max_X,std_X,variance_X,skew_X,min_Y,mean_Y,median_Y,...,variance_Y,skew_Y,min_Z,mean_Z,median_Z,max_Z,std_Z,variance_Z,skew_Z,activity
3340,-5.334281,5.155993,5.111621,21.662737,6.644312,44.146887,0.563811,-10.228029,9.379365,9.129091,...,116.693744,-0.337177,-5.951985,1.842897,0.512359,14.647726,5.338573,28.500360,0.579762,running
1814,-23.381771,1.572991,2.300828,28.874075,11.410129,130.191049,0.146779,-19.321207,11.164641,10.096348,...,219.354726,-0.197772,-25.373749,0.708524,1.254561,17.501614,8.749856,76.559972,-0.659175,running
6436,-12.876017,0.428562,0.646435,9.375694,5.650326,31.926179,-0.528877,-24.823084,-9.069076,-8.681376,...,50.174283,0.170921,-12.287044,-0.826957,-1.175553,13.450625,5.800170,33.641972,0.425217,walking
1700,-9.126697,5.154716,6.291962,26.599580,9.594727,92.058795,0.310629,-12.904747,8.199183,7.141904,...,146.238349,0.188792,-16.883911,-0.247081,-1.618480,14.389153,6.798926,46.225398,0.295941,running
3327,-12.282255,4.368461,3.821147,22.261288,8.452219,71.440011,0.158709,-14.877569,7.088274,7.117962,...,131.657046,0.206350,-12.378023,3.371994,2.039860,35.989640,9.509332,90.427390,1.598983,running


In [11]:
df_new.shape

(6462, 22)

In [12]:
df_new.max(axis=0)

min_X           0.981623
mean_X         13.119427
median_X       10.204087
max_X          39.188293
std_X           15.99138
variance_X    255.724238
skew_X          5.461429
min_Y           6.440403
mean_Y         13.322136
median_Y       15.516821
max_Y          39.188293
std_Y          19.871852
variance_Y    394.890498
skew_Y          5.437522
min_Z           9.744401
mean_Z         10.115183
median_Z         9.77792
max_Z          39.188293
std_Z          13.377245
variance_Z    178.950691
skew_Z          5.228705
activity         walking
dtype: object

### Split DataFrame into random test and train subsets

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = df_new.drop('activity', axis=1)
y = df_new['activity']
y

0       running
1       running
2       running
3       running
4       running
         ...   
6457    walking
6458    walking
6459    walking
6460    walking
6461    walking
Name: activity, Length: 6462, dtype: object

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train

3807       idle
1029    running
5406    walking
3169    running
3971       idle
         ...   
3772       idle
5191    walking
5226    walking
5390    walking
860     running
Name: activity, Length: 4329, dtype: object

In [16]:
scaler = preprocessing.StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### SVM Classifier

In [28]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

In [19]:
# Підбір параметрів for SVM

C_range = [1e-2, 1e-1, 1, 1e1, 1e2]
gamma_range = [1e-1, 1, 1e1]
kernel_p = ['linear', 'poly', 'rbf']
param_grid = dict(gamma=gamma_range, C=C_range, kernel=kernel_p)
random_param_svm = RandomizedSearchCV(svm.SVC(), param_grid, n_iter=10)
random_param_svm.fit(X_train_scaled, y_train)

print("The best parameters for SVM are : ", random_param_svm.best_params_)

The best parameters for SVM are :  {'kernel': 'poly', 'gamma': 1, 'C': 10.0}


In [20]:
classifier = svm.SVC(kernel='poly', gamma=1, C=10.0)

In [21]:
classifier.fit(X_train, y_train)

SVC(C=10.0, gamma=1, kernel='poly')

In [22]:
y_svm_pred = classifier.predict(X_test)
y_svm_pred


array(['running', 'running', 'idle', ..., 'walking', 'running', 'running'],
      dtype=object)

In [23]:
print(classification_report(y_test, y_svm_pred))

              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       335
     running       1.00      1.00      1.00      1144
      stairs       0.91      0.95      0.93        55
     walking       0.99      0.99      0.99       599

    accuracy                           1.00      2133
   macro avg       0.98      0.98      0.98      2133
weighted avg       1.00      1.00      1.00      2133



In [24]:
scores = cross_val_score(classifier, X_train, y_train, cv=10)
scores.mean()

0.9956109400393464

In [25]:
classifier.fit(X_train_scaled, y_train)

SVC(C=10.0, gamma=1, kernel='poly')

In [26]:
y_svm_pred_scaled = classifier.predict(X_test_scaled)

In [29]:
# Підбір параметрів для RandomForestClassifier
rfc=RandomForestClassifier(random_state=42)

In [36]:
param_grid = {
    'n_estimators': [200, 500],
    'max_depth' : [6, 10, 30],
    'criterion' :['gini', 'entropy']
}

In [38]:
CV_rfc = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_grid, n_iter=10, cv= 5)
CV_rfc.fit(X_train_scaled, y_train)

# random_rf = RandomizedSearchCV(RandomForestClassifier(), param_rf, n_iter=10, cv=5)
# random_rf.fit(X_train, y_train)

print("The best parameters for Random Forest are : ", CV_rfc.best_params_)

The best parameters for Random Forest are :  {'n_estimators': 500, 'max_depth': 10, 'criterion': 'entropy'}


In [32]:
# Random Forest
rfc1=RandomForestClassifier(random_state=42, n_estimators= 500, max_depth=10, criterion='entropy')

In [33]:
rfc1.fit(X_train_scaled, y_train)

RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=500,
                       random_state=42)

In [34]:
y_rfm_pred = rfc1.predict(X_test_scaled)

In [ ]:
rfc2=RandomForestClassifier(random_state=42, n_estimators= 500, max_depth=10, criterion='entropy')

In [35]:
print('Model SVM, with normalize')
print(classification_report(y_test, y_svm_pred_scaled), '\n')
print('Model Random Forest, with normalize')
print(classification_report(y_test, y_rfm_pred), '\n')

Model SVM, with normalize
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       335
     running       1.00      1.00      1.00      1144
      stairs       1.00      0.95      0.97        55
     walking       1.00      1.00      1.00       599

    accuracy                           1.00      2133
   macro avg       1.00      0.99      0.99      2133
weighted avg       1.00      1.00      1.00      2133
 

Model Random Forest, with normalize
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       335
     running       1.00      1.00      1.00      1144
      stairs       1.00      0.95      0.97        55
     walking       1.00      1.00      1.00       599

    accuracy                           1.00      2133
   macro avg       1.00      0.99      0.99      2133
weighted avg       1.00      1.00      1.00      2133
 



Висновок: Model SVM і Model Random Forest на моєму наборі ознак для даного датасету видали однаковий результат. До нормалізації результат Model SVM був гіршим, ніж після. Оскільки класи дуже не збалансовані, турбувала саме точність на класі "stairs" - (165 елементів) на противагу класу "running" - (3408 елементів). Очікувала, що Random Forest дасть кращий результат, ніж SVM. Мабуть, треба додавати ще ознаки до набору